In [1]:
import urllib.request, urllib.error, urllib.parse
import requests
import csv
from urllib.request import build_opener, HTTPCookieProcessor, Request
from scholarly import scholarly
import time
import random
import pdfkit

nrf_issn = "1477-8947"
jbe_issn = "1573-0697"
hess_issn = "1607-7938"
test_issn = '2325-1077'

url = 'https://api.crossref.org/journals/'+test_issn+'/works?filter=from-pub-date:2015-01-01&rows=1000'

print(url)

opener = build_opener(HTTPCookieProcessor())
request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = opener.open(request, timeout=30)
allContent = str(response.read())

https://api.crossref.org/journals/2325-1077/works?filter=from-pub-date:2015-01-01&rows=1000


In [2]:
results_i = allContent.find('total-results')
results_i = allContent.find(':',results_i) + 1
results = ""
while(allContent[results_i]!=','):
    results = results + allContent[results_i]
    results_i = results_i+1
print(results,"Total Results")

55 Total Results


In [3]:
print(allContent)

b'{"status":"ok","message-type":"work-list","message-version":"1.0.0","message":{"facets":{},"total-results":55,"items":[{"indexed":{"date-parts":[[2020,8,2]],"date-time":"2020-08-02T14:13:26Z","timestamp":1596377606176},"reference-count":0,"publisher":"Common Ground Research Networks","issue":"3","content-domain":{"domain":[],"crossmark-restriction":false},"short-container-title":["The International Journal of Environmental Sustainability"],"published-print":{"date-parts":[[2017]]},"DOI":"10.18848\\/2325-1077\\/cgp\\/v13i03\\/1-14","type":"journal-article","created":{"date-parts":[[2017,9,8]],"date-time":"2017-09-08T21:04:02Z","timestamp":1504904642000},"page":"1-14","source":"Crossref","is-referenced-by-count":0,"title":["The Case of Brazil\\u2019s Municipal Solid Waste Management: Residents\\u2019 Perceptions"],"prefix":"10.18848","volume":"13","author":[{"given":"Benxiang","family":"Zeng","sequence":"first","affiliation":[]},{"given":"Flavio","family":"de S\\u00e3o Pedro Filho","se

<b>toSheetHeader creates the csv file and its header<b>

In [4]:
def toSheetHeader(name):
    fields = ['Title', 'Abstract','Journal']  
    
    # name of csv file  
    filename = name + " SCRAPED.csv"
      
    with open(filename, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile)   
        csvwriter.writerow(fields)  

<b>toSheet writes a new row to the csv file<b>

In [5]:
def toSheet(rows,name): 
    filename = name + " SCRAPED.csv"
     
    with open(filename, 'a') as csvfile: 
        csvwriter = csv.writer(csvfile)  
        csvwriter.writerows(rows) 

<b>Offset function to get the next page of 1000 results</b>

In [6]:
def offset(i, url):
    url = url + "&offset="+str(i)
    
    print(url)
    
    loop=True
    while(loop):
        try:
            opener = build_opener(HTTPCookieProcessor())
            request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            response = opener.open(request, timeout=30)
            allContent = str(response.read())
            loop=False
        except:
            print("error loading next page")
            
    return(allContent)

<b>findAbstract method returns the abstract index or -1 if not found.</b>

In [7]:
def findAbstract(articleContent):
    abstract_index = articleContent.find('abstractInFull')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div class="abstract">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('Abs1-content')
    if (abstract_index == -1):
        abstract_index = articleContent.find('"abstract-content')
    if (abstract_index == -1):
        abstract_index = articleContent.find('article-section__content')
    if (abstract_index == -1):
        abstract_index = articleContent.find('abstr1')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div class="section1')
    if (abstract_index == -1):
        abstract_index = articleContent.find('section id="abstract"')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<p id="abspara0010">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div id="abssec0010">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<section class="abstract">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div id="as010">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div class="product-specs">')
    if (abstract_index == -1):
        abstract_index = articleContent.find('<div class="mod abstract">')
        
               
    
    return abstract_index

<b>Scholar uses Google Scholar to find the abstract</b>

In [8]:
def scholar(title):
    print("using google scholar")
    
    time.sleep(30) # Sleep for 30 seconds
    
    search_query = scholarly.search_pubs(title)
    scholar = str(next(search_query))
    index = scholar.find("'abstract'") + 13

    abstract = ""
    while(scholar[index]!="'"):
        abstract = abstract + scholar[index]
        index = index + 1
    return(abstract)

<b>Getting Journal Title</b>

In [9]:
journal_index = allContent.find('"container-title"')
journal_index = allContent.find('[',journal_index) + 2
journal = ""
while(allContent[journal_index]!='"'):
    journal = journal + allContent[journal_index]
    journal_index = journal_index+1
print(journal)
toSheetHeader(journal)

The International Journal of Environmental Sustainability


<b>Getting Journal Article Titles and Abstracts</b>

In [ ]:
i = 0
loop = True

while(loop):
    index = 0
    if(i%1000==0 and allContent.find('indexed',index,len(allContent))==-1):
        loop = False
    while(allContent.find('indexed',index,len(allContent))!=-1):
        index = allContent.find('indexed',index,len(allContent))
        title_index = allContent.find('"title":',index,len(allContent))
        title_index = allContent.find('[',title_index) + 2
        title = ""
        while(allContent[title_index]!='"'):
            title = title + allContent[title_index]
            title_index = title_index+1
        
        print(str(i+1)+". "+title)
        
        nexttitle = allContent.find('indexed',title_index,len(allContent))
        if(nexttitle==-1):
            nexttitle=len(allContent)
        
        link = ""
        abstract = ""
        
        if (allContent.find('"abstract"',index,nexttitle)!=-1):
            #if journal supplied abstract to CrossRef
            index = allContent.find('"abstract"',index,nexttitle)+12
            endindex = allContent.find('",',index,nexttitle)
            while(index<endindex):
                abstract = abstract + allContent[index]
                index = index+1
        else:
            #if journal didn't supply, using URL
            first_link = allContent.find('"link"',index,nexttitle)
            if(first_link!=-1):
                index = allContent.find('"URL"',first_link,nexttitle)
            else:
                index = allContent.find('"URL"',index,nexttitle)
            index = allContent.find('http',index)
            while(allContent[index]!='"'):
                link = link + allContent[index]
                index = index+1
            link = link.replace("\\\\","")
            
            if "pdf" in link:
                link = ""
                index = allContent.find('http',index)
                while(allContent[index]!='"'):
                    link = link + allContent[index]
                    index = index+1
                link = link.replace("\\\\","")
                
            if "/xml" in link:
                link = ""
                index = allContent.find('http',index)
                while(allContent[index]!='"'):
                    link = link + allContent[index]
                    index = index+1
                link = link.replace("\\\\","")
            
            tryloop = True
            articleContent = ""
            while tryloop:
                try:
                    randomtime = random.randint(0,100)
                    print("Delaying",randomtime,"Seconds...")
                    time.sleep(randomtime)

                    tryloop = False
                    opener = build_opener(HTTPCookieProcessor())
                    request = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
                    response = opener.open(request, timeout=30)
                    articleContent = str(response.read())
                except:
                    tryloop = True
                    
                    test = allContent.find('"URL"',index,nexttitle)
                    if (test == -1):
                        tryloop = False
                        print("Can't open link.")
                    else:
                        link = ""
                        index = test
                        index = allContent.find('http',index)
                        while(allContent[index]!='"'):
                            link = link + allContent[index]
                            index = index+1
                        link = link.replace("\\\\","")
                    
            
            print(link)
            
            abstract_index = findAbstract(articleContent)
            
            #if(i==1):
                #print(articleContent)
            
            if(abstract_index!=-1):    
                abstract_index = articleContent.find(">",abstract_index,len(articleContent))+2
                x = abstract_index+1
                end_abstract_index = articleContent.find("</p>",abstract_index,len(articleContent))

                abstract = ""

                while(x!=end_abstract_index):
                    abstract = abstract+articleContent[x]
                    x = x + 1
            #else:
                #pdfkit.from_url(link,title+" PDF")
                #print("saved as pdf")
                #except:
                    #print("pdf failed")
            #else:
                #try:
                    #abstract = scholar(title)
                #except:
                    #print("scholar failed")

        abstract = abstract.replace("\\n"," ")
                      
            
        index=index+1
        i = i + 1
        
        
        print(abstract)
        print(str(len(abstract))+"\n\n")
        
        toSheet([[title,abstract,journal]],journal)
        
        
        
    #GETTING NEXT PAGE OF CONTENT IF AVAILABLE  
    if(i%1000==0):
        print("\n")
        allContent = offset(i, url)
        #print(offset)
        print("\n")
    else:
        loop=False


1. The Case of Brazil\\u2019s Municipal Solid Waste Management: Residents\\u2019 Perceptions
Delaying 38 Seconds...
http://dx.doi.org/10.18848/2325-1077/cgp/v13i03/1-14
        <div class="overview"> <h4>Abstract</h4><p>The disposal and treatment of solid waste have been global concerns, especially regarding urban areas. Many people debate the balance between economic development and environment protection. Sustainability is always an issue which attracts attention from not only researchers but also from governmental organizations and the public, either in developed or developing countries. How do the public in developing countries perceive sustainability? Taking the city of Porto Velho in Brazil as an example, this article analyzes the perception of local residents on the management of municipal solid waste to inform governmental decision-making on sustainable practice. Using a sustainability indicator matrix, this study investigates local people\xe2\x80\x99s perceptions of sustainabi